In [1]:
from collections import Counter
import json
from random import shuffle
sahil = None
with open('./data/vrckid.data', 'r') as f:
    sahil = f.read()
    sahil = json.loads(sahil)
others = []
for user in ['_vargas_.data', '-eDgaR-.data', 'Abe_lincolin.data',
             'Ambiguously_Ironic.data', 'anutensil.data', 'APOSTOLATE.data',
            'awildsketchappeared.data', 'axolotl_peyotl.data','boib.data']:
    with open('./data/' + user, 'r') as f:
        other = f.read()
        other = json.loads(other)
        shuffle(other)
        others.extend(other[:120])

In [2]:
import nltk
sahil_char_ngram_counters = Counter()
for comment in sahil:
    for bg in nltk.ngrams(comment, 2):
        sahil_char_ngram_counters[bg] += 1

In [5]:
feature_set = {gram for gram, freq in sahil_char_ngram_counters.most_common(500)}

In [6]:
training_sahil_data = []
for comment in sahil:
    row = [0] * (len(feature_set) + 1)
    ngram_set = set(nltk.ngrams(comment, 2))
    for i, gram in enumerate(feature_set):
        if gram in ngram_set:
            row[i] = 1
    row[-1] = 1
    training_sahil_data.append(row)

training_other_data = []
for comment in others:
    row = [0] * (len(feature_set) + 1)
    ngram_set = set(nltk.ngrams(comment, 2))
    for i, gram in enumerate(feature_set):
        if gram in ngram_set:
            row[i] = 1
    training_other_data.append(row)

In [7]:
print sum([ sum(d) - 1 for d in training_sahil_data])
print sum([ sum(d) for d in training_other_data])

67978
85016


In [8]:
from random import shuffle
shuffle(training_sahil_data)
shuffle(training_other_data)
training = training_sahil_data[:int(len(training_sahil_data) * 0.8)] + training_other_data[:int(len(training_other_data) * 0.8)]
testing = training_sahil_data[int(len(training_sahil_data) * 0.8):] + training_other_data[int(len(training_other_data) * 0.8):]
shuffle(training)
shuffle(testing)
X = [row[:-1] for row in training]
Y = [row[-1] for row in training]

In [9]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()
logreg.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
test_X = [row[:-1] for row in testing]
test_Y = [row[-1] for row in testing]
correct_count = 0
for row, expected in zip(test_X, test_Y):
    predicted = logreg.predict(row)
    if predicted == expected:
        correct_count += 1
print 'Accuracy=', correct_count /float(len(test_X))

Accuracy= 0.720481927711


/Users/Juliang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Juliang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Juliang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single s